In [1]:
from script_general import script_general
import random

In [2]:
#seed = random.randint(0, 1e6)             # Semilla para generar una instancia aleatoria fija.

# semillas malas (gc -> generación de columnas)
# 585 con 10 viajes
# 388
# viajes = 3 , w = 60

lowres = True



K = 0.1                                 # Constante para transformar el consumo de combustible en tiempo recargando.

#num_travels = 10                         # Número de viajes.
num_depots = 3                          # Número de depósitos.
num_charge_stations = 5                 # Número de estaciones de carga.
num_passenger_stations = 10             # Número de estaciones de pasajeros.

w = 100.0                               # Capacidad de combustible de cada vehículo.

speed_bus = 3.0                         # Velocidad del bus (km/h).
fuel_per_distance = 2.0                 # Combustibple requerido por distancia ej. litros de gasolina/km.
cost_per_distance = 1.0                 # Costo por distancia (sucres/km).

h_s = 6.0                               # Hora de inicio de la jornada.
h_f = 20.0                              # Hora de final de la jornada.

time_divisions = 50                     # Número de divisiones temporales para discretización.
factor_low_res = 10                     # para hacer la resolución baja en pasos de factor_low_res 


v_max = 100                             # Capacidad máxima de un depósito dado (más adelante, la capacidad se genera aleatoriamente entre v_min y v_max).
v_min = 10                              # capacidad mínima de un depósito dato


size_square = 10                        # Lado de una ciudad cuadrada (km).


# Se considera la capacidad limitada de las estaciones de carga.
max_capacity_charge_station = 200         # Capacidad máxima de una estacion de carga por unidad de tiempo.
                                        # (mas adelante, la capacidad se genera aleatoriamente entre min_capacity_charge_station y max_capacity_charge_station).

min_capacity_charge_station = 30       # Capacidad mínima de una estación de carga en un tiempo dado  

In [3]:
# experimento solo arcos
'''
viaje_semilla = [(400,111),(400,222)]

for i_viaje, semilla in viaje_semilla:
    seed = semilla
    num_travels = i_viaje
    print(f'------------ {num_travels} --- {seed} ------------')
    script_general(lowres,seed,K,num_travels,num_depots,num_charge_stations,num_passenger_stations,w,speed_bus,fuel_per_distance,cost_per_distance,h_s,h_f,time_divisions,v_max,v_min,size_square,max_capacity_charge_station,min_capacity_charge_station, factor_low_res )
    print("-"*10)

'''

'\nviaje_semilla = [(400,111),(400,222)]\n\nfor i_viaje, semilla in viaje_semilla:\n    seed = semilla\n    num_travels = i_viaje\n    print(f\'------------ {num_travels} --- {seed} ------------\')\n    script_general(lowres,seed,K,num_travels,num_depots,num_charge_stations,num_passenger_stations,w,speed_bus,fuel_per_distance,cost_per_distance,h_s,h_f,time_divisions,v_max,v_min,size_square,max_capacity_charge_station,min_capacity_charge_station, factor_low_res )\n    print("-"*10)\n\n'

In [ ]:
from model_functions import time_discretization, len_trip, distance, estimate_time, cost_fuel
from compatibility_functions import compatibility, comp_T
#from data_paper import data_paper
from graph_construction import create_nodes, create_archs
from plot_functions import plot_city, plot_graph
from generate_data import generate_data
from initial_model import initial_model
from lp_functions import from_V_to_TDS, calculate_cost, calculate_fuel
from export_data import export_data
import random
import gurobipy as gp
import time

#gp.setParam('OutputFlag', 0)


def script_general_ambos_modelos(lowres, seed,K,num_travels,num_depots,num_charge_stations,num_passenger_stations,w,speed_bus,fuel_per_distance,cost_per_distance,h_s,h_f,time_divisions,v_max,v_min,size_square,max_capacity_charge_station,min_capacity_charge_station, factor_low_res ):

    inicio_global = time.time()

    (time_window,
    time_intervals, 
    time_dic, 
    list_time,
    delta,
    list_time_low_res)              = time_discretization(h_s, h_f, time_divisions, factor_low_res, lowres)

    fin1 = time.time()
    print(f"time discretization completed in {fin1 - inicio_global}")

    #Se genera datos para una instancia.
    (Travels, 
    Depots, 
    Stations_chrg, 
    Stations_chrg_time, 
    coord_passenger_stations, 
    coord_depots, 
    coord_charge_stations, 
    T_passenger_stations, 
    T_ab, 
    D_v,
    rs_kt,
    Stations_chrg_time_low_res)                 = generate_data(seed, num_travels, num_depots, num_charge_stations, num_passenger_stations, size_square, v_max, v_min, time_window, max_capacity_charge_station, min_capacity_charge_station, list_time, speed_bus, list_time_low_res, lowres)

    fin2 = time.time()
    print(f"generate data completed in {fin2 - fin1}")

    # Se calcula el tiempo entre estaciones viajes, depósitos y estaciones de carga.
    t                       = estimate_time(Travels, Depots, Stations_chrg, coord_passenger_stations, T_passenger_stations, coord_depots, coord_charge_stations, speed_bus)

    (H_list,
    H_list_inv,
    h, 
    h_inv,
    V,
    V_visual,
    Stations_chrg_time_dic,
    Stations_chrg_time_dic_inv,
    Stations_chrg_time_dic_inv_low_res) = create_nodes(Travels, Depots, Stations_chrg_time, num_travels, num_depots,num_charge_stations, Stations_chrg_time_low_res, lowres )

    fin3 = time.time()
    print(f"create nodes completed in {fin3-fin2}")


    #Se calcula las compatibilidades.
    (dic_comp, 
    dic_comp_F,
    list_comp_FT)            = compatibility(Travels, Depots, Stations_chrg, Stations_chrg_time, T_passenger_stations, T_ab, t, time_dic)

    fin4 = time.time()
    print(f"compatibilities completed in {fin4-fin3}")

    Hk = {k:[h[h_node] for h_node in H_list if h_node[0] in Travels+[k] ] for k in Depots}

    (A1, 
     A2, 
     A3, 
     A4, 
     A5, 
     A6, 
     A7,
     A, 
     A_list,
     delta_mas,
     delta_menos,
     Ak,
     cost,
     fuel,
     A4_prepared,
     A4_dic_preparated,
     Ak_with_i_in_DH, 
     Ak_with_i_not_in_DH, 
     delta_mas_i, 
     delta_mas_k, 
     delta_mas_h)                = create_archs(Travels, Depots, h, H_list, dic_comp, dic_comp_F, T_ab, time_dic, t, Hk, fuel_per_distance, T_passenger_stations, coord_passenger_stations, Stations_chrg, coord_depots, coord_charge_stations, cost_per_distance )

    fin5 = time.time()
    print(f"create archs completed in {fin5-fin4}")

    #Se estiman los costos y el combustible.
   # (cost,
   # fuel)                   = cost_fuel(Travels, Depots, Stations_chrg, coord_passenger_stations, T_passenger_stations, T_ab, coord_depots, coord_charge_stations, h_inv, A, A1, A2, A3, A4, A5, A6, A7, cost_per_distance, fuel_per_distance)
    
    fin6 = time.time()
    print(f"cost and fuel completed in {fin6-fin5}")

    #export_data(num_travels,time_dic,w,seed,V,T_ab,D_v,h_inv,rs_kt,cost,fuel,A1,A2,A3,A4,A5,A6,A7)



    

    print("------------------------------empieza caminos")

    #------------------------------ MODELO BASADO EN CAMINOS
    
    inicio1 = time.time()

    num_max_it          = 1900        #número de máximas iteraciones para el modelo (para no saturar gurobi ni el PC).

    (new_paths, 
    dic_cost_path, 
    x_op,
    z_op)              = initial_model(seed,list_comp_FT,K, Travels, Depots, Stations_chrg_time, Stations_chrg_time_dic, Stations_chrg_time_dic_inv, D_v, rs_kt, h, Stations_chrg, dic_comp, cost, list_time, w, fuel, h_inv, dic_comp_F, T_ab, num_max_it, delta, time_window, t, time_dic, delta_mas_i, delta_mas_k, delta_mas_h, Stations_chrg_time_dic_inv_low_res, lowres)

    fin1 = time.time()
    tiempo_modelo_caminos = fin1 - inicio1
    print(tiempo_modelo_caminos, end='\t')

    print(z_op)




In [ ]:
for seed in [ 345, 456 ]:
    print(seed)
    for i_viaje in [ 360 ]:
        num_travels = i_viaje
        print(f'------------ {num_travels} --- {seed} ------------')
        script_general_ambos_modelos(lowres,seed,K,num_travels,num_depots,num_charge_stations,num_passenger_stations,w,speed_bus,fuel_per_distance,cost_per_distance,h_s,h_f,time_divisions,v_max,v_min,size_square,max_capacity_charge_station,min_capacity_charge_station, factor_low_res )
        print("-"*10)



345
------------ 360 --- 345 ------------
time discretization completed in 2.9802322387695312e-05
generate data completed in 0.001577138900756836
create nodes completed in 0.4244987964630127
compatibilities completed in 27.192976236343384
create archs completed in 30279.35910487175
cost and fuel completed in 0.0003159046173095703
------------------------------empieza caminos
Set parameter OutputFlag to value 1
Set parameter LogFile to value "logs_new/salida_modelo_caminos_360_345.log"
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 613 rows, 1977 columns and 9974 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+10]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
         Consider reformulating model or setting NumericFocus para